In [4]:
import sys
import pymssql
import os
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timedelta
from pathlib import Path
import logging

# Add src to path for imports
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Import ClimateDataFetcher
from src.utils.climate_data import ClimateDataFetcher

load_dotenv()


True

In [5]:
DB_SERVER = os.getenv("DB_SERVER")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

print(f"📡 Servidor: {DB_SERVER}")
print(f"📊 Base de datos: {DB_NAME}")


📡 Servidor: localhost
📊 Base de datos: ClimateDB


# 🔄 Actualización Automática de Base de Datos Climáticos

**Project:** GuajiraClimateAgents  
**Author:** Eder Arley León Gómez  
**GitHub:** https://github.com/ealeongomez  

Este notebook actualiza automáticamente la base de datos `ClimateDB` descargando solo los datos nuevos desde la última fecha registrada hasta hoy.

## Características:
- ✅ Verifica la última fecha por municipio
- ✅ Descarga solo datos faltantes
- ✅ Inserción directa en la base de datos (sin archivos intermedios)
- ✅ Manejo de duplicados
- ✅ Reporte de actualización


In [6]:
conn = pymssql.connect(
    server=DB_SERVER,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    autocommit=True
)

cursor = conn.cursor()
print("✅ Conectado a la base de datos ClimateDB")


✅ Conectado a la base de datos ClimateDB


In [7]:
def get_last_date_per_municipality(cursor):
    """
    Obtiene la última fecha de datos registrada por municipio.
    
    Returns:
        dict: {municipio: última_fecha}
    """
    cursor.execute("""
        SELECT 
            municipio,
            MAX(datetime) as ultima_fecha,
            COUNT(*) as total_registros
        FROM climate_observations
        GROUP BY municipio
        ORDER BY municipio
    """)
    
    result = {}
    print("📅 Última fecha por municipio:")
    print("-" * 70)
    for row in cursor.fetchall():
        municipio, ultima_fecha, total_registros = row
        result[municipio] = ultima_fecha
        print(f"  • {municipio:20s} → {ultima_fecha} ({total_registros:,} registros)")
    
    print("-" * 70)
    return result

# Ejecutar consulta
last_dates = get_last_date_per_municipality(cursor)


📅 Última fecha por municipio:
----------------------------------------------------------------------
  • albania              → 2025-12-18 23:00:00 (87,624 registros)
  • barrancas            → 2025-12-18 23:00:00 (87,624 registros)
  • distraccion          → 2025-12-18 23:00:00 (87,624 registros)
  • el_molino            → 2025-12-18 23:00:00 (87,624 registros)
  • fonseca              → 2025-12-18 23:00:00 (87,624 registros)
  • hatonuevo            → 2025-12-18 23:00:00 (87,624 registros)
  • la_jagua_del_pilar   → 2025-12-18 23:00:00 (87,624 registros)
  • maicao               → 2025-12-18 23:00:00 (87,624 registros)
  • manaure              → 2025-12-18 23:00:00 (87,624 registros)
  • mingueo              → 2025-12-18 23:00:00 (87,624 registros)
  • riohacha             → 2025-12-18 23:00:00 (87,624 registros)
  • san_juan_del_cesar   → 2025-12-18 23:00:00 (87,624 registros)
  • uribia               → 2025-12-18 23:00:00 (87,624 registros)
-----------------------------------------

In [10]:
def bulk_insert_climate_data(cursor, df, municipio):
    """
    Inserta datos climáticos en la base de datos usando MERGE para evitar duplicados.
    
    Args:
        cursor: Cursor de pymssql
        df: DataFrame con los datos a insertar
        municipio: Nombre del municipio
        
    Returns:
        int: Número de registros insertados
    """
    if df.empty:
        print(f"   ⚠️  No hay datos nuevos para {municipio}")
        return 0
    
    # Preparar los datos
    df = df.copy()
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # Asegurar que tenemos todas las columnas necesarias
    required_cols = [
        'datetime', 'wind_speed_10m', 'wind_direction_10m', 
        'temperature_2m', 'relative_humidity_2m', 'precipitation'
    ]
    
    for col in required_cols:
        if col not in df.columns:
            df[col] = None
    
    # Usar MERGE para insertar solo registros nuevos
    inserted = 0
    batch_size = 1000
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        
        # Crear tabla temporal
        cursor.execute("""
            IF OBJECT_ID('tempdb..#TempClimateData') IS NOT NULL
                DROP TABLE #TempClimateData
                
            CREATE TABLE #TempClimateData (
                municipio NVARCHAR(50),
                datetime DATETIME2,
                wind_speed_10m FLOAT,
                wind_direction_10m INT,
                temperature_2m FLOAT,
                relative_humidity_2m INT,
                precipitation FLOAT
            )
        """)
        
        # Insertar datos en tabla temporal
        insert_query = """
            INSERT INTO #TempClimateData 
            (municipio, datetime, wind_speed_10m, wind_direction_10m, 
             temperature_2m, relative_humidity_2m, precipitation)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        
        values = []
        for _, row in batch.iterrows():
            values.append((
                municipio,
                row['datetime'],
                float(row['wind_speed_10m']) if pd.notna(row['wind_speed_10m']) else None,
                int(row['wind_direction_10m']) if pd.notna(row['wind_direction_10m']) else None,
                float(row['temperature_2m']) if pd.notna(row['temperature_2m']) else None,
                int(row['relative_humidity_2m']) if pd.notna(row['relative_humidity_2m']) else None,
                float(row['precipitation']) if pd.notna(row['precipitation']) else None,
            ))
        
        cursor.executemany(insert_query, values)
        
        # Hacer MERGE desde la tabla temporal
        cursor.execute("""
            MERGE climate_observations AS target
            USING #TempClimateData AS source
            ON target.municipio = source.municipio 
               AND target.datetime = source.datetime
            WHEN NOT MATCHED THEN
                INSERT (municipio, datetime, wind_speed_10m, wind_direction_10m,
                        temperature_2m, relative_humidity_2m, precipitation, created_at)
                VALUES (source.municipio, source.datetime, source.wind_speed_10m, 
                        source.wind_direction_10m, source.temperature_2m, 
                        source.relative_humidity_2m, source.precipitation, GETDATE())
            WHEN MATCHED THEN
                UPDATE SET
                    wind_speed_10m = source.wind_speed_10m,
                    wind_direction_10m = source.wind_direction_10m,
                    temperature_2m = source.temperature_2m,
                    relative_humidity_2m = source.relative_humidity_2m,
                    precipitation = source.precipitation;
                    
            SELECT @@ROWCOUNT as affected_rows
        """)
        
        result = cursor.fetchone()
        inserted += result[0] if result else 0
        
        # Limpiar tabla temporal
        cursor.execute("DROP TABLE #TempClimateData")
    
    return inserted

print("✅ Función de inserción masiva definida")

✅ Función de inserción masiva definida


In [11]:
# Obtener lista de municipios
MUNICIPIOS = ClimateDataFetcher.get_available_municipios()

# Fecha actual
NOW = datetime.now()
DEFAULT_START_DATE = datetime(2015, 12, 21)  # Fecha de inicio por defecto si no hay datos

print("🚀 Iniciando actualización de base de datos...")
print("=" * 80)
print(f"📅 Fecha actual: {NOW.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 Municipios a procesar: {len(MUNICIPIOS)}")
print("=" * 80)
print()

# Resultados
results = []
total_inserted = 0
total_downloaded = 0

for idx, municipio in enumerate(MUNICIPIOS, 1):
    print(f"\n[{idx}/{len(MUNICIPIOS)}] 📍 {municipio.upper()}")
    print("-" * 80)
    
    try:
        # Determinar fecha de inicio
        if municipio in last_dates:
            # Si hay datos, empezar desde 1 hora después de la última fecha
            start_date = last_dates[municipio] + timedelta(hours=1)
            print(f"   📅 Última fecha en BD: {last_dates[municipio]}")
            print(f"   📅 Descargando desde: {start_date}")
        else:
            # Si no hay datos, usar fecha por defecto
            start_date = DEFAULT_START_DATE
            print(f"   ⚠️  Sin datos previos. Descargando desde: {start_date}")
        
        # Verificar si necesitamos descargar datos
        if start_date >= NOW:
            print(f"   ℹ️  Base de datos ya está actualizada para {municipio}")
            results.append({
                "municipio": municipio,
                "downloaded": 0,
                "inserted": 0,
                "status": "✅ Ya actualizado"
            })
            continue
        
        # Crear fetcher y descargar datos
        print(f"   ⬇️  Descargando datos...")
        fetcher = ClimateDataFetcher(
            municipio=municipio,
            start_date=start_date,
            end_date=NOW,
            wind_only=False
        )
        
        df = fetcher.fetch(block_days=180)
        
        if df.empty:
            print(f"   ⚠️  No se descargaron datos nuevos para {municipio}")
            results.append({
                "municipio": municipio,
                "downloaded": 0,
                "inserted": 0,
                "status": "⚠️  Sin datos nuevos"
            })
            continue
        
        print(f"   ✅ Descargados {len(df):,} registros")
        total_downloaded += len(df)
        
        # Insertar en base de datos
        print(f"   💾 Insertando en base de datos...")
        inserted = bulk_insert_climate_data(cursor, df, municipio)
        total_inserted += inserted
        
        print(f"   ✅ {inserted:,} registros insertados/actualizados")
        
        results.append({
            "municipio": municipio,
            "downloaded": len(df),
            "inserted": inserted,
            "start_date": df['datetime'].min(),
            "end_date": df['datetime'].max(),
            "status": "✅ Actualizado"
        })
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        results.append({
            "municipio": municipio,
            "downloaded": 0,
            "inserted": 0,
            "status": f"❌ Error: {str(e)[:50]}"
        })

print("\n" + "=" * 80)
print("✅ ACTUALIZACIÓN COMPLETADA")
print("=" * 80)
print(f"📥 Total descargado: {total_downloaded:,} registros")
print(f"💾 Total insertado: {total_inserted:,} registros")
print()


2026-01-02 14:23:35,887 - INFO - Descargando archivo: 2025-12-19 a 2026-01-01 para albania


🚀 Iniciando actualización de base de datos...
📅 Fecha actual: 2026-01-02 14:23:35
📊 Municipios a procesar: 13


[1/13] 📍 ALBANIA
--------------------------------------------------------------------------------
   📅 Última fecha en BD: 2025-12-18 23:00:00
   📅 Descargando desde: 2025-12-19 00:00:00
   ⬇️  Descargando datos...


2026-01-02 14:23:38,117 - INFO - Descargando datos recientes para albania
2026-01-02 14:23:38,944 - INFO - Total de registros descargados: 360


   ✅ Descargados 360 registros
   💾 Insertando en base de datos...


2026-01-02 14:23:40,174 - INFO - Descargando archivo: 2025-12-19 a 2026-01-01 para barrancas


...

   ✅ Descargados 360 registros
   💾 Insertando en base de datos...


2026-01-02 14:24:24,584 - INFO - Descargando archivo: 2025-12-19 a 2026-01-01 para uribia


   ✅ 360 registros insertados/actualizados

[13/13] 📍 URIBIA
--------------------------------------------------------------------------------
   📅 Última fecha en BD: 2025-12-18 23:00:00
   📅 Descargando desde: 2025-12-19 00:00:00
   ⬇️  Descargando datos...


2026-01-02 14:24:26,445 - INFO - Descargando datos recientes para uribia
2026-01-02 14:24:27,214 - INFO - Total de registros descargados: 360


   ✅ Descargados 360 registros
   💾 Insertando en base de datos...
   ✅ 360 registros insertados/actualizados

✅ ACTUALIZACIÓN COMPLETADA
📥 Total descargado: 4,680 registros
💾 Total insertado: 4,680 registros



In [12]:
# Crear DataFrame con resultados
df_results = pd.DataFrame(results)

# Estadísticas
successful = df_results[df_results["status"].str.contains("✅")]
failed = df_results[df_results["status"].str.contains("❌")]

print("📊 RESUMEN FINAL")
print("=" * 80)
print(f"✅ Municipios actualizados: {len(successful)}/{len(MUNICIPIOS)}")
print(f"❌ Municipios con errores: {len(failed)}")
print(f"📥 Total descargado: {total_downloaded:,} registros")
print(f"💾 Total insertado en BD: {total_inserted:,} registros")
print()

# Mostrar tabla
df_results


📊 RESUMEN FINAL
✅ Municipios actualizados: 13/13
❌ Municipios con errores: 0
📥 Total descargado: 4,680 registros
💾 Total insertado en BD: 4,680 registros



,municipio,downloaded,inserted,start_date,end_date,status
0,albania,360,360,2025-12-19,2026-01-02 23:00:00,✅ Actualizado
1,barrancas,360,360,2025-12-19,2026-01-02 23:00:00,✅ Actualizado
2,distraccion,360,360,2025-12-19,2026-01-02 23:00:00,✅ Actualizado
3,el_molino,360,360,2025-12-19,2026-01-02 23:00:00,✅ Actualizado
4,fonseca,360,360,2025-12-19,2026-01-02 23:00:00,✅ Actualizado
5,hatonuevo,360,360,2025-12-19,2026-01-02 23:00:00,✅ Actualizado
6,la_jagua_del_pilar,360,360,2025-12-19,2026-01-02 23:00:00,✅ Actualizado
7,maicao,360,360,2025-12-19,2026-01-02 23:00:00,✅ Actualizado
8,manaure,360,360,2025-12-19,2026-01-02 23:00:00,✅ Actualizado
9,mingueo,360,360,2025-12-19,2026-01-02 23:00:00,✅ Actualizado


In [13]:
# Verificar estado actualizado de la base de datos
cursor.execute("""
    SELECT 
        municipio,
        MIN(datetime) as fecha_inicio,
        MAX(datetime) as ultima_fecha,
        COUNT(*) as total_registros
    FROM climate_observations
    GROUP BY municipio
    ORDER BY municipio
""")

print("📊 ESTADO ACTUAL DE LA BASE DE DATOS")
print("=" * 90)
print(f"{'Municipio':<25} {'Fecha Inicio':<20} {'Última Fecha':<20} {'Registros':>15}")
print("-" * 90)

total_records = 0
for row in cursor.fetchall():
    municipio, fecha_inicio, ultima_fecha, registros = row
    total_records += registros
    print(f"{municipio:<25} {str(fecha_inicio):<20} {str(ultima_fecha):<20} {registros:>15,}")

print("-" * 90)
print(f"{'TOTAL':<25} {'':<20} {'':<20} {total_records:>15,}")
print("=" * 90)


📊 ESTADO ACTUAL DE LA BASE DE DATOS
Municipio                 Fecha Inicio         Última Fecha               Registros
------------------------------------------------------------------------------------------
albania                   2015-12-21 00:00:00  2026-01-02 23:00:00           87,984
barrancas                 2015-12-21 00:00:00  2026-01-02 23:00:00           87,984
distraccion               2015-12-21 00:00:00  2026-01-02 23:00:00           87,984
el_molino                 2015-12-21 00:00:00  2026-01-02 23:00:00           87,984
fonseca                   2015-12-21 00:00:00  2026-01-02 23:00:00           87,984
hatonuevo                 2015-12-21 00:00:00  2026-01-02 23:00:00           87,984
la_jagua_del_pilar        2015-12-21 00:00:00  2026-01-02 23:00:00           87,984
maicao                    2015-12-21 00:00:00  2026-01-02 23:00:00           87,984
manaure                   2015-12-21 00:00:00  2026-01-02 23:00:00           87,984
mingueo                   2015-12

In [15]:
# Cerrar conexión
cursor.close()
conn.close()

print("✅ Conexión cerrada correctamente")
print("\n🎉 Actualización de base de datos completada exitosamente!")


✅ Conexión cerrada correctamente

🎉 Actualización de base de datos completada exitosamente!
